# Sentiment Analysis of Tourist Reviews  
### Data Preprocessing & Machine Learning Project  
**Datasets used:**  
- Yelp Academic Review Dataset  
- Yelp Check-in Dataset

In [15]:
import pandas as pd
import numpy as np
import json

chunk_size = 1000000
chunks = pd.read_json("C:/Users/DELL 7540/Downloads/Compressed/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json", lines=True, chunksize=chunk_size)

reviews_sample = pd.DataFrame()
for i, chunk in enumerate(chunks):
    if i < 7:
        reviews_sample = pd.concat([reviews_sample, chunk])
    else:
        break

reviews = reviews_sample.copy()
reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [22]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [25]:
reviews.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [27]:
reviews.duplicated().sum()

0

In [33]:
reviews = reviews[['review_id', 'business_id', 'stars', 'text']]
reviews

,review_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,jals67o91gcrD4DC81Vk6w,5,Latest addition to services from ICCU is Apple...
6990276,shTPgbgdwTHSuU67mGCmZQ,2vLksaMmSEcGbjI5gywpZA,5,"This spot offers a great, affordable east week..."
6990277,YNfNhgZlaaCO5Q_YJR4rEw,R1khUUxidqfaJmcpmGd4aw,4,This Home Depot won me over when I needed to g...
6990278,i-I4ZOhoX70Nw5H0FwrQUA,Rr9kKArrMhSLVE9a53q-aA,5,For when I'm feeling like ignoring my calorie-...


In [35]:
reviews.dropna(subset=['text'])
reviews.dropna(subset=['stars'])

,review_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,jals67o91gcrD4DC81Vk6w,5,Latest addition to services from ICCU is Apple...
6990276,shTPgbgdwTHSuU67mGCmZQ,2vLksaMmSEcGbjI5gywpZA,5,"This spot offers a great, affordable east week..."
6990277,YNfNhgZlaaCO5Q_YJR4rEw,R1khUUxidqfaJmcpmGd4aw,4,This Home Depot won me over when I needed to g...
6990278,i-I4ZOhoX70Nw5H0FwrQUA,Rr9kKArrMhSLVE9a53q-aA,5,For when I'm feeling like ignoring my calorie-...


In [37]:
reviews.drop_duplicates(subset=['review_id'])

,review_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,jals67o91gcrD4DC81Vk6w,5,Latest addition to services from ICCU is Apple...
6990276,shTPgbgdwTHSuU67mGCmZQ,2vLksaMmSEcGbjI5gywpZA,5,"This spot offers a great, affordable east week..."
6990277,YNfNhgZlaaCO5Q_YJR4rEw,R1khUUxidqfaJmcpmGd4aw,4,This Home Depot won me over when I needed to g...
6990278,i-I4ZOhoX70Nw5H0FwrQUA,Rr9kKArrMhSLVE9a53q-aA,5,For when I'm feeling like ignoring my calorie-...


In [47]:
duplicate_count = reviews['text'].duplicated().sum()
print(f"Duplicate found: {duplicate_count}")

Duplicate found: 16153


In [76]:
reviews= reviews.drop_duplicates(subset=['text'])

In [78]:
reviews

,review_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,jals67o91gcrD4DC81Vk6w,5,Latest addition to services from ICCU is Apple...
6990276,shTPgbgdwTHSuU67mGCmZQ,2vLksaMmSEcGbjI5gywpZA,5,"This spot offers a great, affordable east week..."
6990277,YNfNhgZlaaCO5Q_YJR4rEw,R1khUUxidqfaJmcpmGd4aw,4,This Home Depot won me over when I needed to g...
6990278,i-I4ZOhoX70Nw5H0FwrQUA,Rr9kKArrMhSLVE9a53q-aA,5,For when I'm feeling like ignoring my calorie-...


In [80]:
duplicate_count = reviews['text'].duplicated().sum()
print(f"Duplicate found: {duplicate_count}")

Duplicate found: 0


In [92]:
print(f"business_id missing after: {reviews['business_id'].isnull().sum()}")

business_id missing after: 0


In [93]:
print(f"review_id missing after: {reviews['review_id'].isnull().sum()}")

review_id missing after: 0


In [95]:
print(f"stars missing after: {reviews['stars'].isnull().sum()}")

stars missing after: 0


In [98]:
print(f"text missing after: {reviews['text'].isnull().sum()}")

text missing after: 0


In [90]:
reviews.to_csv("reviews.csv", index=False)
